In [8]:
import base64
import httpx
from langchain_core.messages import HumanMessage

from langchain_openai import ChatOpenAI, OpenAIEmbeddings

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")
message = HumanMessage(
    content=[
        {"type": "text", "text": "Liste em texto absolutamente todas as comidas na imagem."},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ]
)

In [9]:
model_name = "gpt-4o-2024-08-06"

llm = ChatOpenAI(model=model_name, temperature=0.01)

response = llm.invoke([message])

In [10]:
from pprint import pprint

pprint(response.content)

('A imagem não contém comidas. É uma paisagem com um caminho de madeira '
 'cercado por grama e vegetação sob um céu azul.')


In [ ]:
from pydantic import BaseModel, Field
from typing import List
from langchain_groq import ChatGroq

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model_name_groq = "llama-3.2-90b-text-preview"
model_name_openai = "gpt-4o-2024-08-06"

llm_openai = ChatOpenAI(
    model=model_name_openai, # 100% json output
    temperature=0,
)

llm_groq = ChatGroq(
    model=model_name_groq, 
    temperature=0,
)

system_prompt = """

Você é um assistente de nutrição que auxilia na listagem de alimentos dado um texto que descreve uma imagem. Registre em uma lista absolutamente todas as comidas presentes no texto.

"""

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt), 
            ("human", "Descrição da imagem: \n\n {query}")
        ]
)

class FoodList(BaseModel):
    """Gere uma lista de alimentos presentes no texto."""
    
    resultado: List[str] = Field(description="Lista de alimentos presentes na descrição da cena.", 
                                 examples=[["Banana", "Maçã", "Pão", "Queijo"],
                                           ["Arroz", "Feijão", "Carne", "Salada"]])
    

llm_openai_with_tools_extraction = llm_openai.bind_tools([FoodList]) #, strict=True)
llm_groq_with_tools_extraction = llm_groq.with_structured_output(FoodList)

chain_openai_structured_extraction = prompt | llm_openai_with_tools_extraction
chain_groq_structured_extraction = prompt | llm_groq_with_tools_extraction

NameError: name 'GetSchema' is not defined